## Make Profile summaries using Jinja2 and Python Modele

*USE Python 3.7 to maintain order of Json files*

- Fetch SD file from IG
- Spec_internals from IG Package.tgz
- Create and use import Title map from ?Package?
- Transform to Python model
- use Jinja2 template to create a summary markdown file
- save markdown file



*Note need a successful build to generate since based on ig output local file
alternatively use package.files to generate*

### import python modules including R4 fhirclient models

In [71]:
from fhirclient.r4models import structuredefinition as SD
from fhirclient.r4models import narrative as N
from fhirclient.r4models import valueset as VS
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
import fhirclient.r4models.fhirreference as FR
from json import dumps, loads, load
from pprint import pprint
from jinja2 import Environment, FileSystemLoader, select_autoescape
from commonmark import commonmark
from IPython.display import display, HTML, Markdown
import title_map as tm
import os
from stringcase import snakecase, titlecase
from pathlib import Path

### Get file and return as dict

In [72]:
#ig_package_path =  "//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/output"
ig_package_path =  "/Users/ehaas/Documents/FHIR/US-Core-R4/output"
#ig_source_path = "//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/source/"
ig_source_path = "/Users/ehaas/Documents/FHIR/US-Core-R4/source/"
#ig_source_path = ''

def open_file(in_path, f_name): # get files
    with open(f'{in_path}/{f_name}', encoding="utf8") as f:
        r = f.read()
        return(loads(r))
   

### Get spec_internal from package.tgz a json file which includes canonical to local relative page links

In [73]:
import tarfile
from pprint import pprint
def get_si(path):
    with tarfile.open(name=os.path.join(path,'package.tgz'), mode='r') as tf:
        #pprint(tf.getnames())
        f = tf.extractfile('other/spec.internals')
        r = f.read()
        return(loads(r))
        
si = get_si(ig_package_path)
path_map = si['paths']
c = 'http://hl7.org/fhir/us/core/ValueSet/birthsex'
path_map[c]


'ValueSet-birthsex.html'

### Using Jinja2 Template create md file for summary view 
*using Markdown instead of html for easier hand editing, the line spacing is resolved after template is rendered.*

uses these elements:
 -   'label',
 -   'short',
 -  'min',
 -  'max',
 -   'type',
 -   'binding',
 
 plus:
 an invariant list
 a hash table of value set urls to valueset titles.


In [74]:
def get_summary(profile_id,diff,constraints):

    in_path = ''
    template_path = 'summary-template.j2'
    core_path = 'http://hl7.org/fhir/R4/'

    bindings = dict(
        required = f'{core_path}terminologies.html#required',
        extensible = f'{core_path}terminologies.html#extensible',
        preferred =f'{core_path}terminologies.html#preferred',
        example = f'{core_path}terminologies.html#example',
    )

    def markdown(text, *args, **kwargs):
        return commonmark(text, *args, **kwargs)

    env = Environment(
        loader=FileSystemLoader(searchpath = in_path),
        autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
        )

    env.filters['markdown'] = markdown

    template = env.get_template(template_path)
    d = template.render(elements = diff, title_map=tm.title_map, bindings=bindings, constraints=constraints, path_map = path_map)

    print(f'============== file_name = {profile_id}-summary.md ================')
    display(Markdown(d))
    return d

### Loop through profiles and update missing stuff in Differential with Snapshot then generate Markdown summary file and save

In [75]:
#in_path = '/Users/ehaas/Documents/FHIR/US-Core-R4/output/StructureDefinition-'
#f_name = 'us-core-patient'
#in_path = '/Users/ehaas/.fhir/packages/hl7.fhir.us.core.r4#dev/package/' # package file keeps disappearing
in_path = '/Users/ehaas/Documents/FHIR/US-Core-R4/output'  # use local build dir.
# in_path ='//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/output'  # when working on PC
#files = [x for x in os.listdir(in_path) if x.startswith("StructureDefinition") and x.endswith('json')]
files = [x for x in os.listdir(in_path) if x.startswith("StructureDefinition") and x.endswith('json')]
files

['StructureDefinition-us-core-observation-lab.json',
 'StructureDefinition-us-core-careteam.json',
 'StructureDefinition-us-core-medicationstatement.json',
 'StructureDefinition-us-core-direct.json',
 'StructureDefinition-us-core-profile-link.json',
 'StructureDefinition-us-core-device.json',
 'StructureDefinition-us-core-practitioner.json',
 'StructureDefinition-us-core-patient.json',
 'StructureDefinition-us-core-immunization.json',
 'StructureDefinition-us-core-smokingstatus.json',
 'StructureDefinition-us-core-documentreference.json',
 'StructureDefinition-us-core-race.json',
 'StructureDefinition-us-core-medicationrequest.json',
 'StructureDefinition-us-core-condition.json',
 'StructureDefinition-us-core-birthsex.json',
 'StructureDefinition-us-core-encounter.json',
 'StructureDefinition-us-core-organization.json',
 'StructureDefinition-us-core-pediatric-weight.json',
 'StructureDefinition-us-core-procedure.json',
 'StructureDefinition-us-core-medication.json',
 'StructureDefiniti

In [76]:
summ_elements =[
       'label',
       'short',
       'min',
       'max',
       'type',
       'binding',
       'sliceName',
        ]

choice_types = {'valueQuantity': 'value[x]',
                'valueCodeableConcept': 'value[x]',
                'valueString': 'value[x]',
                'valueInteger': 'value[x]',
                'valueDecimal': 'value[x]',
                'valueDateTime': 'value[x]',
                'valueRange': 'value[x]',
                'valuePeriod': 'value[x]',
                'dueDuration': 'due[x]',
                'dueDate': 'due[x]',
                'effectivedateTime': 'effective[x]',
                'effectivePeriod': 'effective[x]',
               } 

In [77]:
#files = [i for i in files if i =='StructureDefinition-us-core-practitioner.json']
for i in files:
    
    constraints = {}
    sd_dict = open_file(in_path,i)
    sd = SD.StructureDefinition(sd_dict)
    profile_id = sd.id
    print(f'========{profile_id}=======')
    for i in sd.differential.element:
        path = i.path
        #print(f'====Path = {path} =====')
        # GET Invariant Dict  path: human readable invariant list.
        try:
            k = next((k for k in sd.snapshot.element if k.path == path))
            constraint = [j.human for j in k.constraint if 'dom-' not in j.key and 'ele-' not in j.key  and 'ext-' not in j.key]
        except TypeError:
            constraint = []
        except StopIteration:
            constraint = []
        if constraint:
            constraints[path]=constraint         

        for k in summ_elements:
            #print(f'differential = {path}.{k} = {getattr(i,k)}')
            if getattr(i,k) == None:
                try:
                    snap_element = (s for s in sd.snapshot.element if s.path == path)           
                    new_val = getattr(next(snap_element),k)
                    #print(f'snapshot = {path}.{k} = {new_val}')
                    setattr(i,k,new_val)
                except StopIteration: # assume is an choice data type
                    print(f'no snapshot element for {path}.{k} = {getattr(i,k)} assume is an choice data type')
                    new_plist = []
                    for p in path.split('.'):
                        try:
                            new_plist.append(choice_types[p])
                        except KeyError:
                            new_plist.append(p)
                    new_path = '.'.join(new_plist)
                    #print(path,new_path)
                    snap_element = (s for s in sd.snapshot.element if s.path == new_path)           
                    new_val = getattr(next(snap_element),k)
                    #print(f'snapshot = {path}.{k} = {new_val}')
                    setattr(i,k,new_val)
            #print(f'differential post if = {path}.{k} = {getattr(i,k)}')
    summ_file = get_summary(profile_id,sd.differential.element,constraints)
    summ_file = os.linesep.join([s for s in summ_file.splitlines() if s]) # remove empty lines
    summ_file = summ_file.replace('####', '\n####')  #add a line before Must Supports

    f_name = f'{profile_id}-summary.md'
    # save in pages folder
    #ig_source_path = ''  # temp folder
    path = Path.cwd() / ig_source_path / 'pages' / '_includes' / f_name
    path.write_text(summ_file, encoding='utf8')

========us-core-observation-lab=======
============== file_name = us-core-observation-lab-summary.md ================




**Observation**
#### Summary of the Mandatory Requirements


1.  A  code  in `Observation.status`
with a [required](http://hl7.org/fhir/R4/terminologies.html#required)
 binding to [ObservationStatus](http://hl7.org/fhir/ValueSet/observation-status)
1. One or more  CodeableConcepts  in `Observation.category`
with a [preferred](http://hl7.org/fhir/R4/terminologies.html#preferred)
 binding to [Observation Category Codes](http://hl7.org/fhir/ValueSet/observation-category) with the following constraints: *Must have a category of &#39;laboratory&#39; and a code system &#39;http://hl7.org/fhir/observation-category&#39;*
1.  A  CodeableConcept  in `Observation.code`
with an [extensible](http://hl7.org/fhir/R4/terminologies.html#extensible)
 binding to [LOINC Codes](http://hl7.org/fhir/ValueSet/observation-codes)
1.  A Patient Reference  in `Observation.subject`

#### Summary of the Must Support Requirements


1.  A  dateTime  in `Observation.effective[x]`
 with the following constraints: *Datetime must be at least to day.*
1.  A  Quantity  in `Observation.value[x]`
 with the following constraints: *SHOULD use Snomed CT for coded Results, SHALL use UCUM for coded quantity units.*
1.  A  CodeableConcept  in `Observation.dataAbsentReason`
with an [extensible](http://hl7.org/fhir/R4/terminologies.html#extensible)
 binding to [DataAbsentReason](http://hl7.org/fhir/ValueSet/data-absent-reason)
#### Summary of Constraints
1. If Observation.code is the same as an Observation.component.code then the value element associated with the code SHALL NOT be present
1. dataAbsentReason SHALL only be present if Observation.value[x] is not present
1. If there is no component or related element then either a value[x] or a data absent reason must be present
1. Must have a category of &#39;laboratory&#39; and a code system &#39;http://hl7.org/fhir/observation-category&#39;
1. Datetime must be at least to day.
1. SHOULD use Snomed CT for coded Results
1. SHALL use UCUM for coded quantity units.

========us-core-careteam=======
============== file_name = us-core-careteam-summary.md ================




**CareTeam**
#### Summary of the Mandatory Requirements


1.  A Patient Reference  in `CareTeam.subject`

1. One or more  Participants  in `CareTeam.participant`
 with the following constraints: *CareTeam.participant.onBehalfOf can only be populated when CareTeam.participant.member is a Practitioner*
      - which must have a  CodeableConcept value  in `CareTeam.participant.role`
with an [extensible](http://hl7.org/fhir/R4/terminologies.html#extensible)\+ [MaxValueSet](general-guidance.html#max-binding)
 binding to [CareTeam Provider Roles](ValueSet-us-core-careteam-provider-roles.html)
      - which must have a Member Reference value  in `CareTeam.participant.member`

#### Summary of the Must Support Requirements


1.  A  code  in `CareTeam.status`
with a [required](http://hl7.org/fhir/R4/terminologies.html#required)
 binding to [CareTeamStatus](http://hl7.org/fhir/ValueSet/care-team-status)
#### Summary of Constraints
1. CareTeam.participant.onBehalfOf can only be populated when CareTeam.participant.member is a Practitioner

========us-core-medicationstatement=======
============== file_name = us-core-medicationstatement-summary.md ================


...



**Observation**
#### Summary of the Mandatory Requirements


1.  A  CodeableConcept  in `Observation.code`
with an [extensible](http://hl7.org/fhir/R4/terminologies.html#extensible)
 binding to [Vital Signs](http://hl7.org/fhir/ValueSet/observation-vitalsignresult)
      - which must have one or more  Coding values  in `Observation.code.coding`

      - which must have at least  a  Coding value  in `Observation.code.coding`

         - which must have a fixed `Observation.code.coding.system` = `http://loinc.org`
         - which must have a fixed `Observation.code.coding.code` = `59576-9`
1.  A  Quantity  in `Observation.valueQuantity`

      - which must have a  decimal value  in `Observation.valueQuantity.value`

      - which must have a  string value  in `Observation.valueQuantity.unit`

      - which must have a fixed `Observation.valueQuantity.system` = `http://unitsofmeasure.org`
      - which must have a fixed `Observation.valueQuantity.code` = `%`


#### Summary of Constraints
1. If Observation.code is the same as an Observation.component.code then the value element associated with the code SHALL NOT be present
1. dataAbsentReason SHALL only be present if Observation.value[x] is not present
1. If there is no component or hasMember element then either a value[x] or a data absent reason must be present.

========us-core-diagnosticreport-lab=======
============== file_name = us-core-diagnosticreport-lab-summary.md ================




**DiagnosticReport**
#### Summary of the Mandatory Requirements


1.  A  code  in `DiagnosticReport.status`
with a [required](http://hl7.org/fhir/R4/terminologies.html#required)
 binding to [DiagnosticReportStatus](http://hl7.org/fhir/ValueSet/diagnostic-report-status)
1.  A  CodeableConcept  in `DiagnosticReport.category`
with an [example](http://hl7.org/fhir/R4/terminologies.html#example)
 binding to [Diagnostic Service Section Codes](http://hl7.org/fhir/ValueSet/diagnostic-service-sections)
1.  A  CodeableConcept  in `DiagnosticReport.code`
with an [extensible](http://hl7.org/fhir/R4/terminologies.html#extensible)
 binding to [](ValueSet-us-core-diagnosticreport-lab-codes.html)
1.  A Patient Reference  in `DiagnosticReport.subject`

1.  A  dateTime  in `DiagnosticReport.effective[x]`

1.  An  instant  in `DiagnosticReport.issued`

#### Summary of the Must Support Requirements


1. One or more Performer References  in `DiagnosticReport.performer`

1. One or more Result References  in `DiagnosticReport.result`
 with the following constraints: *DiagnosticReport.result or  DiagnosticReport.presentedForm or both SHALL be present.*
1. One or more  Medias  in `DiagnosticReport.media`

1. One or more  Attachments  in `DiagnosticReport.presentedForm`
 with the following constraints: *DiagnosticReport.result or  DiagnosticReport.presentedForm or both SHALL be present.*
#### Summary of Constraints
1. DiagnosticReport.result or  DiagnosticReport.presentedForm or both SHALL be present.
1. DiagnosticReport.result or  DiagnosticReport.presentedForm or both SHALL be present.
1. DiagnosticReport.result or  DiagnosticReport.presentedForm or both SHALL be present.

========us-core-allergyintolerance=======
============== file_name = us-core-allergyintolerance-summary.md ================




**AllergyIntolerance**
#### Summary of the Mandatory Requirements


1.  A  CodeableConcept  in `AllergyIntolerance.code`
with an [extensible](http://hl7.org/fhir/R4/terminologies.html#extensible)
 binding to [Common substances for allergy and intolerance documentation including refutations](https://vsac.nlm.nih.gov/valueset/2.16.840.1.113762.1.4.1186.8/expansion)
1.  A Patient Reference  in `AllergyIntolerance.patient`

#### Summary of the Must Support Requirements


1.  A  CodeableConcept  in `AllergyIntolerance.clinicalStatus`
with a [required](http://hl7.org/fhir/R4/terminologies.html#required)
 binding to [AllergyIntolerance Clinical Status Codes](http://hl7.org/fhir/ValueSet/allergyintolerance-clinical)
1.  A  CodeableConcept  in `AllergyIntolerance.verificationStatus`
with a [required](http://hl7.org/fhir/R4/terminologies.html#required)
 binding to [AllergyIntolerance Verification Status Codes](http://hl7.org/fhir/ValueSet/allergyintolerance-verification)
#### Summary of Constraints
1. AllergyIntolerance.clinicalStatus SHALL be present if verificationStatus is not entered-in-error.
1. AllergyIntolerance.clinicalStatus SHALL NOT be present if verification Status is entered-in-error